In [ ]:
import collections
import numpy as np
import json
import os
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import measurements
import skimage.exposure
from sklearn import linear_model
from typing import Any, Dict, Iterable, List, Mapping, Optional, Tuple
import gdal
import gdalconst
import copy
import pandas as pd
from PIL import Image

In [ ]:
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes

In [ ]:
from matplotlib import image


In [ ]:
from scipy import ndimage

In [ ]:
from google.colab import drive # m.p. added cell, this cell mounts your google drive to the colab
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
# The code in this cell takes the JSON files with the bounding box coordinates and swaps x2, y1, and writes the classifier, boudning box coordinates
#filename and width, height 

bbox_path = '/content/gdrive/MyDrive/json_image_files/json_files_with_bbox_data'
yolo_path = '/content/gdrive/MyDrive/yoloV5_format_file'


for filename in os.listdir(bbox_path):
  f = open(os.path.join(bbox_path,filename))
  indiv_json_file = json.loads(f.read()) # reads file in as a python object
  list_of_polygons = indiv_json_file['polygons']
  num_of_contrails_infile = (len(list_of_polygons))
  current_contrail = 0
  image_size = 1400
  classifier = np.array([0])

  while current_contrail < num_of_contrails_infile:
    #print(list_of_polygons[current_contrail][-1])  #verification
    if len(list_of_polygons[current_contrail][-1]) < 4:
      print(filename)
      current_contrail = current_contrail + 1
      continue
    #perform swap
    x2 = list_of_polygons[current_contrail][-1][1]
    y1 = list_of_polygons[current_contrail][-1][2] 
    temp = copy.deepcopy(list_of_polygons[current_contrail][-1][1]) # store x2 in temp
    x2 = copy.deepcopy(y1)
    y1 = copy.deepcopy(temp)
    list_of_polygons[current_contrail][-1][1] = x2
    list_of_polygons[current_contrail][-1][2] = y1
    #print(list_of_polygons[current_contrail][-1]) # verification

    fileType = '.txt'
    textfile_name_list = filename.split(".") 
    textfile_name = textfile_name_list[0]+ fileType

    contrail1 = np.array(list_of_polygons[current_contrail][-1]) # convert list to a numpy arry
    contrail1 = contrail1/image_size  # normalize bounding box
    #print(contrail1) # verification 
    y = open(os.path.join(yolo_path,textfile_name), 'a') # open text file
    classifier = list(classifier)
    contrail1 = list(contrail1)
    for num in classifier:
      y.write(str(num))
      y.write(" ")
    for number in contrail1:
      y.write(str(number))
      y.write(" ")
    y.write(" \n")
    current_contrail = current_contrail + 1
    
    #print(" ")
  #print(current_contrail)
  

In [ ]:
    # sample file names to test code
                                          LC08_L1TP_034022_20180729_20180814_01_T1_B10

                                          LC08_L1TP_032037_20180816_20180829_01_T1_B10

                                          LC08_L1TP_025037_20180308_20180320_01_T1_B10

                                          LC08_L1TP_032037_20180816_20180829_01_T1_B10

                                          LC08_L1TP_006028_20180709_20180717_01_T1_B10

                                          LC08_L1TP_011048_20181117_20181129_01_T1_B10

In [ ]:
#This cell opens an Landsat-8 image png  that has polygon annotations and converts it to 3 channel RBG, the images from landsat-8 are 4 channel because they have alpha
img = Image.open(('/content/gdrive/MyDrive/tif_image_files/annotated_images_with_contrails/LC08_L1TP_011048_20181117_20181129_01_T1_B10.png')).convert('RGB')

In [ ]:

#save the converted RGB image
img.save('img.jpeg')

In [ ]:
print(img.width) # check the width 
print(img.height)# check the height

1440
1440


In [ ]:
img = img.resize((1440, 1470)) # resize the image to keep aspect ratio of the annotated image

In [ ]:
print(img.width)# verify the resize worked
print(img.height)# verify the resize worked
img.save('img.jpeg') # save the resized image

1440
1470


In [ ]:

img = read_image('/content/img.jpeg') # read in the resized image as a tensor



In [ ]:
# This cell is a testing if bounding boxes enclose the polygon annotations, since this is a test, not all boxes are assigned from the polygons
json_file = '/content/gdrive/MyDrive/json_image_files/json_files_with_bbox_data/LC08_L1TP_011048_20181117_20181129_01_T1_B10.json'

f = open('/content/gdrive/MyDrive/json_image_files/json_files_with_bbox_data/LC08_L1TP_011048_20181117_20181129_01_T1_B10.json')
indiv_json_file = json.loads(f.read()) # reads file in as a python object
list_of_polygons = indiv_json_file['polygons']
num_of_contrails_infile = (len(list_of_polygons))
print(num_of_contrails_infile) # check how many contrails in the scene
one_box0 = list_of_polygons[0][-1]
one_box1 = list_of_polygons[1][-1]
one_box2 = list_of_polygons[2][-1]

one_box3 = list_of_polygons[3][-1]
one_box4 = list_of_polygons[4][-1]

one_box5 = list_of_polygons[5][-1]

# not checking these polygons since this is just a test, only checking the 1st 6 contrails
"""
one_box6 = list_of_polygons[6][-1]
one_box7 = list_of_polygons[7][-1]
one_box8 = list_of_polygons[8][-1]
one_box9 = list_of_polygons[9][-1]
one_box10 = list_of_polygons[10][-1]
one_box11 = list_of_polygons[11][-1]
one_box12 = list_of_polygons[12][-1]
one_box13 = list_of_polygons[13][-1]
one_box14 = list_of_polygons[14][-1]
"""

# Assigning the xmin, xmax, ymin, ymax for 1st 6 polygons in scene
# you MUST run the image name through the Landsat-8_contrails code to get the original image size
# each image is a different width and height

b0 = np.array(one_box0) #(X1[0], X2[1], Y1[2], Y2[3])
b0 = [(b0[0]/763)*1440, (b0[1]/763)*1440, (b0[2]/779)*1440, (b0[3]/779)*1440]

b1 = np.array(one_box1)
b1 = [(b1[0]/763)*1440, (b1[1]/763)*1440, (b1[2]/779)*1440, (b1[3]/779)*1440]

b2 = np.array(one_box2)
b2 = [(b2[0]/763)*1440, (b2[1]/763)*1440, (b2[2]/779)*1440, (b2[3]/779)*1440]

b3 = np.array(one_box3)
b3 = [(b3[0]/763)*1440, (b3[1]/763)*1440, (b3[2]/779)*1440, (b3[3]/779)*1440]

b4 = np.array(one_box4)
b4 = [(b4[0]/763)*1440, (b4[1]/763)*1440, (b4[2]/779)*1440, (b4[3]/779)*1440]

b5 = np.array(one_box5)
b5 = [(b5[0]/763)*1440, (b5[1]/763)*1440, (b5[2]/779)*1440, (b5[3]/779)*1440]

"""
b0 = b0.tolist()
b1 = b1.tolist()
b2 = b2.tolist()
b3 = b3.tolist()
b4 = b4.tolist()
b5 = b5.tolist()
"""
x0, y0 ,w0, h0 = b0
x1, y1, w1, h1 = b1
x2, y2, w2, h2 = b2

x3, y3, w3, h3 = b3
x4, y4, w4, h4 = b4
x5, y5, w5, h5 = b5
"""
x6, y6, w6, h6 = one_box6
x7, y7, w7, h7 = one_box7
x8, y8, w8, h8 = one_box8
x9, y9, w9, h9 = one_box9

x10, y10, w10, h10 = one_box10
x11, y11, w11, h11 = one_box11
x12, y12, w12, h12 = one_box12
x13, y13, w13, h13 = one_box13
x14, y14, w14, h14 = one_box14

"""
bbox0 = [x0, w0, y0, h0]
bbox1 = [x1, w1, y1, h1]
bbox2 = [x2, w2, y2, h2]

bbox3 = [x3, w3, y3, h3]
bbox4 = [x4, w4, y4, h4]
bbox5 = [x5, w5, y5,h5]
"""
bbox6 = [x6, w6, y6, h6]
bbox7 = [x7, w7, y7, h7]
bbox8 = [x8, w8, y8, h8]
bbox9 = [x9, w9, y9, h9]
bbox10 = [x10, w10, y10, h10]
bbox11 = [x11, w11, y11, h11]
bbox12 = [x12, w12, y12, h12]
bbox13 = [x13, w13, y13, h13]
bbox14 = [x14, w14, y14, h4]
"""


bbox0 = torch.tensor(bbox0, dtype=torch.float32)
bbox1 = torch.tensor(bbox1, dtype=torch.float32)
bbox2 = torch.tensor(bbox2, dtype=torch.float32)

bbox3 = torch.tensor(bbox3, dtype=torch.float32)
bbox4 = torch.tensor(bbox4, dtype=torch.float32)
bbox5 = torch.tensor(bbox5, dtype=torch.float32)

"""
bbox6 = torch.tensor(bbox6, dtype=torch.int)
bbox7 = torch.tensor(bbox7, dtype=torch.int)
bbox8 = torch.tensor(bbox8, dtype=torch.int)
bbox9 = torch.tensor(bbox9, dtype=torch.int)
bbox10 = torch.tensor(bbox10, dtype=torch.int)
bbox11 = torch.tensor(bbox11, dtype=torch.int)
bbox12 = torch.tensor(bbox12, dtype=torch.int)
bbox13 = torch.tensor(bbox13, dtype=torch.int)
bbox14 = torch.tensor(bbox14, dtype=torch.int)



13


"\nbbox6 = torch.tensor(bbox6, dtype=torch.int)\nbbox7 = torch.tensor(bbox7, dtype=torch.int)\nbbox8 = torch.tensor(bbox8, dtype=torch.int)\nbbox9 = torch.tensor(bbox9, dtype=torch.int)\nbbox10 = torch.tensor(bbox10, dtype=torch.int)\nbbox11 = torch.tensor(bbox11, dtype=torch.int)\nbbox12 = torch.tensor(bbox12, dtype=torch.int)\nbbox13 = torch.tensor(bbox13, dtype=torch.int)\nbbox14 = torch.tensor(bbox14, dtype=torch.int)\n\n#image1 = cv2.imread('/content/gdrive/MyDrive/tif_image_files/images_with_contrails/LC08_L1TP_021030_20180224_20180308_01_T1_B10.png')\n\ncv2.rectangle(image1, (int(x0),int(y0)), (int(w0), int(h0)),(0,0,255),-1)\nimage1 = np.array(image1)\ncv2_imshow(image1)\ncv2.rectangle\n"

In [ ]:
# print the normalized x and y coordinates of bounding box
print (bbox0)
print (bbox1)
print (bbox2)

print (bbox3)
print (bbox4)
print (bbox5)
"""
print (bbox6)
print (bbox7)
print (bbox8)
print (bbox9)
print (bbox10)
print (bbox11)
print (bbox12)
print (bbox13)
print (bbox14)
"""

tensor([ 586.2668,  927.3120,  816.4215, 1000.8277])
tensor([136.3754, 617.9800, 178.6695, 844.5350])
tensor([ 347.4684, 1174.6628,  421.5067, 1200.2650])
tensor([ 616.9541,  966.4082,  871.4359, 1026.4298])
tensor([789.8485, 839.9323, 864.2642, 850.4318])
tensor([ 419.6760, 1178.2306,  505.9250, 1202.3723])


'\nprint (bbox6)\nprint (bbox7)\nprint (bbox8)\nprint (bbox9)\nprint (bbox10)\nprint (bbox11)\nprint (bbox12)\nprint (bbox13)\nprint (bbox14)\n'

In [ ]:




bbox0 = bbox0.unsqueeze(0)
bbox1 = bbox1.unsqueeze(0)
bbox2 = bbox2.unsqueeze(0)

bbox3 = bbox3.unsqueeze(0)
bbox4 = bbox4.unsqueeze(0)
bbox5 = bbox5.unsqueeze(0)
"""
bbox6 = bbox6.unsqueeze(0)
bbox7 = bbox7.unsqueeze(0)
bbox8 = bbox8.unsqueeze(0)
bbox9 = bbox9.unsqueeze(0)
bbox10 = bbox10.unsqueeze(0)
bbox11 = bbox11.unsqueeze(0)
bbox12 = bbox12.unsqueeze(0)
bbox13 = bbox13.unsqueeze(0)
bbox14 = bbox14.unsqueeze(0)
"""

'\nbbox6 = bbox6.unsqueeze(0)\nbbox7 = bbox7.unsqueeze(0)\nbbox8 = bbox8.unsqueeze(0)\nbbox9 = bbox9.unsqueeze(0)\nbbox10 = bbox10.unsqueeze(0)\nbbox11 = bbox11.unsqueeze(0)\nbbox12 = bbox12.unsqueeze(0)\nbbox13 = bbox13.unsqueeze(0)\nbbox14 = bbox14.unsqueeze(0)\n'

In [ ]:
# draw red bounding box on image, that should enclose the annotations
img = draw_bounding_boxes(img, bbox0, width=2, colors=(255,255,0))
img = draw_bounding_boxes(img, bbox1, width=2, colors=(255,255,0))
img = draw_bounding_boxes(img, bbox2, width=2, colors=(255,255,0))

img = draw_bounding_boxes(img, bbox3, width=1, colors=(255,255,0))
img = draw_bounding_boxes(img, bbox4, width=1, colors=(255,255,0))
img = draw_bounding_boxes(img, bbox5, width=1, colors=(255,255,0))
"""
img = draw_bounding_boxes(img, bbox6, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox7, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox8, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox9, width=1, colors=(255,0,0))

img = draw_bounding_boxes(img, bbox10, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox11, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox12, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox13, width=1, colors=(255,0,0))
img = draw_bounding_boxes(img, bbox14, width=1, colors=(255,0,0))
"""

'\nimg = draw_bounding_boxes(img, bbox6, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox7, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox8, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox9, width=1, colors=(255,0,0))\n\nimg = draw_bounding_boxes(img, bbox10, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox11, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox12, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox13, width=1, colors=(255,0,0))\nimg = draw_bounding_boxes(img, bbox14, width=1, colors=(255,0,0))\n'

In [ ]:
# Transform image to PIL image
img = torchvision.transforms.ToPILImage()(img)
img.show()

In [ ]:
#save PIL image
img.save('img0.jpeg')